In [3]:
import sys
#change the next line to reflect where you have downloaded the source code
sys.path.insert(0, './SBDynT-main/src')
import sbdynt as sbd

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
merged_df = pd.read_csv("../data/nesvorny_catalog_dataset.csv")
des = merged_df["Des'n"].to_list()[-10000:]

/tmp/ipykernel_14739/4191221711.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv("../data/nesvorny_catalog_dataset.csv")


In [7]:
from astroquery.jplsbdb import SBDB

e_sig_list = []
i_sig_list = []

for i in range(10000):
	sbdb = SBDB.query(des[i])
	e_sig_list.append(sbdb['orbit']['elements']['e_sig'])
	i_sig_list.append(sbdb['orbit']['elements']['i_sig'])

In [8]:
query_df = pd.DataFrame(list(zip(des, e_sig_list)), columns = ["Des'n", "e_sig"])
query_df = query_df[query_df["e_sig"] > 0.002]

In [9]:
high_error_list = query_df["Des'n"].to_list()

In [10]:
# example with 5 clones, 
# the first index on the returned variables is best fit, 
# followed by 5 clones sampled from the covariance matrix
clones = 100
columns = ["Des'n", "epoch", "x", "y", "z", "vx", "vy", "vz"]
df = pd.DataFrame(columns=columns)
for name in high_error_list[:100]:
	flag, epoch, x,y,z,vx,vy,vz, weights  = sbd.query_sb_from_jpl(des=name,clones=clones)
	df1 = []
	if(flag):
		print("queried %s and returned at epoch %f" % (name,epoch))
		print("cartesian heliocentric position (au), velocity (au/year)")
		print("best-fit orbit:")
		i=0
		print(6*"%15.8e " % (x[i],y[i],z[i],vx[i],vy[i],vz[i]))
		print("cloned orbits:")
		for j in range (1,clones):
			df1.append({
				"Des'n": name,
				"epoch": epoch,
				"x": x[j], 
				"y": y[j], 
				"z": z[j], 
				"vx": vx[j], 
				"vy": vy[j], 
				"vz": vz[j]
			})
	df1 = pd.DataFrame(df1)
	df = pd.concat([df, df1])

queried K23Q65H and returned at epoch 2460193.500000
cartesian heliocentric position (au), velocity (au/year)
best-fit orbit:
 2.08691674e+00 -4.70991121e-01 -5.05396854e-02  1.12255761e+00  4.37539626e+00 -1.60271126e-01 
cloned orbits:


/tmp/ipykernel_14739/1901823227.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df1])


queried K23Q67X and returned at epoch 2460180.500000
cartesian heliocentric position (au), velocity (au/year)
best-fit orbit:
 2.10300691e+00 -1.96323402e+00 -1.99311038e-01  2.62326571e+00  2.79062870e+00  3.00106468e-01 
cloned orbits:
queried K23Q75M and returned at epoch 2460194.500000
cartesian heliocentric position (au), velocity (au/year)
best-fit orbit:
 2.67716313e+00 -2.26263815e-01 -1.27126829e-01  3.84163712e-01  3.95055615e+00  7.62065130e-01 
cloned orbits:
queried K23Q76N and returned at epoch 2460197.500000
cartesian heliocentric position (au), velocity (au/year)
best-fit orbit:
 2.32296839e+00 -2.08145561e-01 -3.38929819e-02  1.04886907e+00  4.24860883e+00  2.19710039e-01 
cloned orbits:
horizons_api.query_sb_from_jpl failed
WARNING!!! The object's observational arc is less than 30 days which probably
means the orbit is of too low quality for useful dynamical analysis and it's not
possible to produce useful clones. This object can be re-run, but only for
clones=0 and e

In [13]:
df = df[df["x"]!= 0]
df.to_csv("./tables_for_analysis/uncertainty_asteroids_sampled.csv")
len(df)

3465